___

#### Group 2:
#### DJ Dawkins, Rick Fontenot, Joe Lazarus, Puri Rudick
___


# Lab 2: Classification

___
# Data Preparation
#### [15 points total] 
___

> 
> ##   Data Prep - Define and Prepare Your Class Variables 
> 
Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

> #### [10 points]

>
> ## Data Prep - Describe The Final Dataset that is used for classification/Regression
>
(include a description of any newly formed variables you created).

> #### [5 points]

# Modeling and Evaluation 


#### [70 points total]

> 
> ## Choose and explain your evaluation metrics
> 

i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

> #### [10 points]

> 
> ## Choose the method you will use for dividing your data into training and testing splits
> 

i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate.

> #### [10 points]

> 
> ## Create three different classification/regression models (e.g., random forest, KNN, and SVM).
> 

Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization

> #### [20 points]

>
> ## Analyze the results using your chosen method of evaluation.
> 

Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

#### [10 points]

>
> ## Discuss the advantages of each model for each classification task
> 

(if any) If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods.

> #### [10 points]

>
> ## Which attributes from your analysis are most important?
> 

Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

> #### [10 points]

# Deployment 

How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?

#### [5 points total]

# Exceptional Work 

You have free reign to provide additional modeling. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

#### [10 points total]